# K-Nearest Neighbors (KNN) from Scratch

**Author:** Gemini
**Date:** September 5, 2025
**Location:** Delhi, India

This notebook provides a step-by-step implementation of the **K-Nearest Neighbors (KNN)** algorithm from scratch using Python. We will use the Breast Cancer Wisconsin dataset to classify tumors as either Malignant or Benign. 

## Understanding K-Nearest Neighbors

The K-nearest neighbors (KNN) algorithm is a type of supervised machine learning algorithm. KNN is extremely easy to implement in its most basic form, and yet performs quite complex classification tasks. It is a **lazy learning** algorithm since it doesn't have a specialized training phase. Rather, it uses all of the data for training while classifying a new data point or instance. KNN is a **non-parametric** learning algorithm, which means that it doesn't assume anything about the underlying data.

### Pros

- It is extremely easy to implement.
- It is a lazy learning algorithm and therefore requires no training prior to making real-time predictions.
- New data can be added seamlessly.
- There are only two parameters required: the value of K and the distance function (e.g., Euclidean).

### Cons

- Doesn't work well with high-dimensional data (curse of dimensionality).
- Has a high prediction cost for large datasets.
- Doesn't work well with categorical features.

In [6]:
# Import necessary libraries for data manipulation, numerical operations, and helper functions.
import pandas as pd
import numpy as np
import math
import operator  # Used for sorting items in a dictionary

## Step 1: Data Loading and Preprocessing

First, we load the dataset from a URL, inspect it, and then preprocess it by converting categorical labels to numbers and dropping unnecessary columns.

In [7]:
# URL of the raw CSV file from a GitHub repository.
url = 'https://raw.githubusercontent.com/melwinlobo18/K-Nearest-Neighbors/master/Dataset/data.csv'
# Read the dataset into a pandas DataFrame.
df = pd.read_csv(url)

print("--- Original Data (First 5 Rows) ---")
display(df.head())

# Convert the categorical 'diagnosis' column ('M'/'B') to numerical labels (1 for Malignant, 2 for Benign).
df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 2})
# Create a new column named 'Class' which will be our target variable.
df['Class'] = df['diagnosis'].tolist()

# Drop unnecessary columns: 'id', 'Unnamed: 32' (empty), and the original 'diagnosis' column.
df = df.drop(['id', 'Unnamed: 32', 'diagnosis'], axis=1)

print("\n--- Processed Data (First 5 Rows) ---")
display(df.head())

--- Original Data (First 5 Rows) ---


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN



--- Processed Data (First 5 Rows) ---


,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Class
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,1
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,1
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,1
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,1
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,1


## Step 2: Train-Test Split

We will manually split our dataset into a training set (70%) and a testing set (30%). The model will learn from the training data, and we will evaluate its performance on the unseen test data.

In [8]:
# Set a seed for the random number generator to ensure the split is the same every time.
np.random.seed(1)

# Create a boolean mask: an array of True/False values. Approx. 70% of values will be True.
msk = np.random.rand(len(df)) < 0.7

# Use the mask to select ~70% of the data for the training set.
train = df[msk]
# Use the inverted mask (~) to select the remaining ~30% for the test set.
test = df[~msk]

# Print the number of observations in each set to verify the split.
print(f'Number of observations in the training data: {len(train)}')
print(f'Number of observations in the test data: {len(test)}')

Number of observations in the training data: 395
Number of observations in the test data: 174


## Step 3: Building the KNN Model from Scratch

We will now define the core functions that make up our KNN classifier.

### 3a. Euclidean Distance

The most common distance metric used for KNN is the Euclidean distance. It's the straight-line distance between two points in the feature space.

In [9]:
def euclideanDistance(instance1, instance2, length):
    """Calculates the Euclidean distance between two data instances."""
    distance = 0
    # Iterate through each feature.
    for x in range(length):
        # Add the squared difference of the feature values to the distance.
        distance += pow((instance1[x] - instance2[x]), 2)
    # Return the square root of the sum of squared differences.
    return math.sqrt(distance)

### 3b. Get Neighbors

This function finds the `k` most similar instances (the nearest neighbors) from the training set for a given test instance. **This is the cell that has been completed.**

In [10]:
def getNeighbors(trainingSet, testInstance, k):
    """Finds the k nearest neighbors for a test instance."""
    distances = []
    length = len(testInstance) - 1

    # Calculate the distance from the test instance to every training instance.
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
        
    # Sort the distances list in ascending order based on the distance value.
    distances.sort(key=operator.itemgetter(1))

    neighbors = []
    # Get the top 'k' instances from the sorted list.
    for x in range(k):
        neighbors.append(distances[x][0])
        
    return neighbors

### 3c. Get Response (Prediction by Voting)

Once we have the k-nearest neighbors, we need to determine the predicted class for our test instance. This is done by a majority vote: the class that appears most frequently among the neighbors is chosen as the prediction.

In [11]:
def getResponse(neighbors):
    """Gets the majority vote from a list of neighbors to predict the class."""
    classVotes = {}
    # Loop through each neighbor.
    for x in range(len(neighbors)):
        # The last element (-1) of a neighbor instance is its class label.
        response = neighbors[x][-1]
        if response in classVotes:
            classVotes[response] += 1  # If the class is already a key, increment its count.
        else:
            classVotes[response] = 1   # If not, add it to the dictionary with a count of 1.
            
    # Sort the dictionary by the vote count in descending order.
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    # Return the class label with the highest count.
    return sortedVotes[0][0]

### 3d. Get Accuracy

This function calculates the accuracy of our predictions by comparing them to the true labels of the test set.

In [12]:
def getAccuracy(testSet, predictions):
    """Calculates the accuracy of the predictions."""
    correct = 0
    # Loop through each instance in the test set.
    for x in range(len(testSet)):
        # Check if the actual label (last element) matches the predicted label.
        if testSet[x][-1] == predictions[x]:
            correct += 1
    # Calculate accuracy: (Correct Predictions / Total Predictions) * 100.
    return (correct / float(len(testSet))) * 100.0

## Step 4: Making Predictions

Now we tie everything together. We'll iterate through our test set, find the neighbors for each instance, get a prediction, and store it.

In [13]:
# List to store the final predicted values.
predictions = []
# Set the number of nearest neighbors to 3.
k = 3
# Convert the DataFrames to lists of lists for easier processing.
trainingSet = train.values.tolist()
testSet = test.values.tolist()

# Loop through each instance in the test set to make a prediction.
print("Running Predictions...")
for x in range(len(testSet)):
    neighbors = getNeighbors(trainingSet, testSet[x], k)
    result = getResponse(neighbors)
    predictions.append(result)
    # Print the predicted class vs. the actual class for each test instance.
    # print('> predicted=' + repr(result) + ', actual=' + repr(testSet[x][-1]))
print("Predictions Complete.")

Running Predictions...
Predictions Complete.


## Step 5: Evaluating the Model

Finally, we'll evaluate our model's performance using two common metrics: the **Confusion Matrix** and **Accuracy Score**.

In [14]:
# Import evaluation metrics from scikit-learn.
from sklearn.metrics import confusion_matrix, accuracy_score

# --- Confusion Matrix ---
print("--- Confusion Matrix ---")
# Create a list of the true labels from the test set.
y_test = test['Class'].tolist()

# Calculate and print the confusion matrix.
conf_matrix = confusion_matrix(y_test, predictions)
print(conf_matrix)

# --- Accuracy ---
# Calculate the accuracy of our predictions.
accuracy = accuracy_score(y_test, predictions) * 100
# Print the final accuracy percentage.
print(f'\nAccuracy: {accuracy:.2f}%')

--- Confusion Matrix ---
[[ 52   5]
 [  6 111]]

Accuracy: 93.68%
